In [1]:
#import argparse
#import os
from plyfile import PlyData, PlyElement
import numpy as np
from sklearn.decomposition import PCA
import pptk

In [19]:
import os
import numpy as np
import argparse
#import laspy
import math
from tqdm import tqdm
import random 
import shutil
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

import open3d as o3d
#import pyntcloud
import pandas as pd
#from pyntcloud import Voxelize
from scipy.spatial import cKDTree
import time

parser = argparse.ArgumentParser()

"""
parser.add_argument('--segment_point_size', type=int, default=131072, help='resample points number') #Choose here number of point resample
parser.add_argument('--dataset_type', type=str, choices = ('train', 'test'), default='train', help='if train, dataset just splited, downsampled, normalized, if test same, but also gives numbers to reconstruct and paint dataset later')
parser.add_argument('--output_dataset_folder', type=str, default=r'nuevaparticion/test/', help='train datasetfolder') #las_test
"""

#parser.add_argument('--original_las', type=str, default=r'raws/train/', help='train datasetfolder') #las_test
#parser.add_argument('--dataset_type', type=str, choices = ('train', 'test'), default='train', help='if train, dataset just splited, downsampled, normalized, if test same, but also gives numbers to reconstruct and paint dataset later')
#opt = parser.parse_args()


#opt.original_las

def split_dataset(output_folder, opt_original_las):
    print("Procesando nubes...")

    # Ruta de la carpeta para guardar los segmentos
    #output_folder = opt.output_dataset_folder

    # Crear la carpeta si no existe
    #os.makedirs(output_folder, exist_ok=True)

    for filename in tqdm(os.listdir(opt_original_las), desc ="Conjunto de Todas las nubes:"):
        f = os.path.join(opt_original_las, filename)
        if os.path.isfile(f):

            print(f)

            plydata = PlyData.read(f) #print(plydata)

            x = plydata["vertex"].data["x"].astype(np.float32)
            y = plydata["vertex"].data["y"].astype(np.float32)
            z = plydata["vertex"].data["z"].astype(np.float32)

            point_data = np.stack([x, y, z], axis=0)

            #ESTO SI HAY QUE RESTAR 1 A LAS CLASESS
            #adapta_clases = las.classification
            #adapta_clases = np.subtract(adapta_clases, 1)

            reflectance = plydata["vertex"].data["reflectance"].astype(np.float32)

            label = plydata["vertex"].data["class"].astype(np.float32) #label solo si es train
            
            #true_label = plydata["vertex"].data["label"].astype(np.float32) #label solo si es train
            #reflectance = label = plydata["vertex"].data["reflectance"].astype(np.float32) #label solo si es train

            xyz = np.array([x,y,z]).T
            
            xyzic = np.stack([x, y, z, reflectance , label], axis=0).transpose((1, 0))
            print(xyzic.shape)


            ###################

            x_min = np.min(xyzic[:, 0])
            x_max = np.max(xyzic[:, 0])
            y_min = np.min(xyzic[:, 1])
            y_max = np.max(xyzic[:, 1])

            divide_x = np.ceil((x_max - x_min) / 51).astype(int)
            divide_y = np.ceil((y_max - y_min) / 51).astype(int)

            count = 0
            for i in range(divide_x):
                for j in range(divide_y):
                    # Definir límites de segmento
                    x_min_segment = x_min + i * 51
                    x_max_segment = x_min + (i + 1) * 51
                    y_min_segment = y_min + j * 51
                    y_max_segment = y_min + (j + 1) * 51

                    # Filtrar puntos dentro del segmento
                    segment_points = xyzic[
                        (xyzic[:, 0] >= x_min_segment)
                        & (xyzic[:, 0] < x_max_segment)
                        & (xyzic[:, 1] >= y_min_segment)
                        & (xyzic[:, 1] < y_max_segment)
                    ]
                    count += 1

                    filename_seg = os.path.splitext(filename)[0]+f"_{str(count).zfill(3)}"
                    #filename = f"segment_{i}_{j}.npy"

                    filepath = os.path.join(output_folder, filename_seg)
                    if len(segment_points)>1:
                        np.save(filepath, segment_points)


        




In [ ]:
opt_original_las = r'C:/Users/sfernandez/nueva_etapa/github2/LidarSegmentationPytorch/Datasets/ParisLille/training_10_classes'
output_folder = r'train/'
os.makedirs(output_folder, exist_ok=True)

split_dataset(output_folder, opt_original_las)

AHORA A PROCESAR.
